<a href="https://colab.research.google.com/github/antoinexp/markov-chains-COM-516/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is provided as a starting point to help you generate random instances G1 and G2 as mentioned in the handout.

You are free to use and modify it at your own convenience.

---



In [ ]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from time import time
import pandas as pd

In [ ]:
class DatasetGenerator(object):
    def __init__(self, N=100):
        self.N = N
        self.x = None
        self.v = None
        self.refresh()

    def refresh(self):
        raise Exception("undefined")

In [ ]:
class G1(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = st.uniform().rvs((self.N,))

In [ ]:
class G2(DatasetGenerator):
    def refresh(self):
        self.x = st.uniform().rvs((self.N,2))
        self.v = np.exp(st.norm(-0.85, 1.3).rvs((self.N,)))

### Uniform distribution ($\mathcal U([0,1])$)

In [ ]:
g1 = G1(100)

Examples:

In [ ]:
# Plot a histogram of the v array
plt.hist(g1.v, bins=30)
plt.show()
type(g1.v)

In [ ]:
# plot the position of the points
# plt.figure(figsize=(5,5))
sns.scatterplot(g1.x[:,0], g1.x[:,1], hue=g1.v)
plt.show()

You can refresh the dataset

In [ ]:
g1.refresh() # generate a new dataset

In [ ]:
plt.hist(g1.v, bins=30)
plt.show()

Or for instance, you can generate 10 instances and compute the average position of all the points:

In [ ]:
m = np.array([0., 0.])

for _ in range(10):
  g1.refresh() # refresh the dataset
  m  += 0.1*g1.x.mean()

print(m)

### Test on log-normal distribution

In [ ]:
g2 = G2()

Example:

you can use g2 to generate an instance of the lognormal distribution

In [ ]:
plt.hist(g2.v, bins=30)
plt.show()

In [ ]:
# plot the position of the points
# plt.figure(figsize=(5,5))
sns.scatterplot(g2.x[:,0], g2.x[:,1], hue=g2.v)
plt.show()

In [ ]:
g2.refresh() # to generate a new x and v

In [ ]:
plt.hist(g2.v, bins=30)
plt.show()

---

### Metropolis Hastings

In [ ]:
# fill-in this section with your code

from SimulatedAnnealingtris import SimulatedAnnealing as Sim_an3
from SimulatedAnnealing import SimulatedAnnealing as Sim_an1
from SimulatedAnnealingbis import SimulatedAnnealing as Sim_an2

In [ ]:
iterations = 3000
cycles = 10
lambda_ = 1
dataset = g1
print(g1.x.shape)

In [ ]:
def create_data_model(dataset):
    """Stores the data for the problem."""
    cities = dataset.x
    dist =  np.zeros((dataset.N, dataset.N))
    radius = np.inf
    center = None
    min_dist = np.inf
    min_i = None
    min_j = None
    max_dist = 0
    max_i = None
    max_j = None
    for i in range(dataset.N):
      r = np.linalg.norm(cities[i] - [0.5, 0.5])  # or most populous
      if r < radius:
        radius = r
        center = i
      for j in range(dataset.N):
        dist[i, j] = np.linalg.norm(cities[i] - cities[j])
        if dist[i, j] < min_dist and dist[i, j] != 0:
          min_dist = dist[i, j]
          min_i = i
          min_j = j
        if dist[i, j] > max_dist:
          max_dist = dist[i, j]
          max_i = i
          max_j = j
    print(cities[center])

    data = {}
    data['distance_matrix'] = np.round(dist / min_dist)
    data['num_vehicles'] = 1
    data['depot'] = center
    data['min_dist'] = min_dist
    data['min_i'] = min_i
    data['min_j'] = min_j
    data['max_dist'] = max_dist
    data['max_i'] = max_i
    data['max_j'] = max_j
    return data

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue() * data['min_dist']))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

# Instantiate the data problem.
data = create_data_model(dataset)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], [data['min_i']], [data['min_j']])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

SA = Sim_an3(lambda_, dataset, order = get_routes(solution, routing, manager)[0])
# print(SA.S.get_objective())
# SA.cool_down(iterations)
# print(SA.S.get_objective())
SA.heat_cool_cycles(iterations,cycles)
plt.plot(range(iterations*cycles+1), SA.objectives)
plt.show()
SA.plot_betas()

In [ ]:
# iterations = 2000
# cycles = 10
SA = Sim_an2(lambda_, dataset)
# print(SA.S.get_objective())
# SA.cool_down(iterations)
# print(SA.S.get_objective())
SA.heat_cool_cycles(iterations,cycles)
plt.plot(range(iterations*cycles+1), SA.objectives)
plt.show()
SA.plot_betas()

In [ ]:
# SA = Sim_an1(0, g1,0.1)
# print(SA.S.get_objective())
# SA.cool_down(1000)
# print(SA.S.get_objective())
# plt.plot(range(1001), SA.objectives)
# plt.show()


In [ ]:
runs = 10
lmbds = 3
lambdas = np.linspace(0, 1, lmbds)
results = pd.DataFrame(['run', '$\lambda$', 'version', 'objective', 'cardinality'])
for i in range(runs):
    dataset = g1.refresh()
    for j, lmbd in enumerate(lambdas):
        # -------------------------------------------------- Sim_an1 --------------------------------------------------
        SA = Sim_an1(lmbd, dataset)
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 1, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

        # -------------------------------------------------- Sim_an2 --------------------------------------------------
        SA = Sim_an2(lmbd, dataset)
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 2, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

        # -------------------------------------------------- Sim_an3 --------------------------------------------------
        # Instantiate the data problem.
        data = create_data_model(dataset)

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], [data['min_i']], [data['min_j']])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        SA = Sim_an3(lmbd, dataset, order = get_routes(solution, routing, manager)[0])
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 3, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

sns.lineplot(data = results, x = '$\lambda$', y = 'objective', hue = 'version')
plt.show()
sns.lineplot(data = results, x = '$\lambda$', y = 'cardinality', hue = 'version')
plt.show()


In [ ]:
lambdas = np.linspace(0, 2, lmbds)
results = pd.DataFrame(['run', '$\lambda$', 'version', 'objective', 'cardinality'])
for i in range(runs):
    dataset = g2.refresh()
    for j, lmbd in enumerate(lambdas):
        # -------------------------------------------------- Sim_an1 --------------------------------------------------
        SA = Sim_an1(lmbd, dataset)
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 1, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

        # -------------------------------------------------- Sim_an2 --------------------------------------------------
        SA = Sim_an2(lmbd, dataset)
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 2, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

        # -------------------------------------------------- Sim_an3 --------------------------------------------------
        # Instantiate the data problem.
        data = create_data_model(dataset)

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], [data['min_i']], [data['min_j']])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        SA = Sim_an3(lmbd, dataset, order = get_routes(solution, routing, manager)[0])
        SA.heat_cool_cycles(iterations, cycles)
        results.append({'run': i, '$\lambda$': j, 'version': 3, 'objective': SA.S.get_objective(), 'cardinality': SA.S.assignments.sum()}, ignore_index = True)

sns.lineplot(data = results, x = '$\lambda$', y = 'objective', hue = 'version')
plt.show()
sns.lineplot(data = results, x = '$\lambda$', y = 'cardinality', hue = 'version')
plt.show()